## Homework9 SGD experiments
**Prepare for Data Generation**

In [1]:
import numpy as np
from matplotlib import pyplot as plt
# Specify m, n
m = 500
n = 50
# Specify the mean and variance of noise \delta, which follows normal distribution.
mu = 0
# This can be changed.
sigma = 0.8
np.random.seed(1234)

In [2]:
# define a function to generate experimental data.
# Input args: m,n are matrix shape; mu and sigma are for noise data.
def data_generator(m,n,mu, sigma):
    # A is a matrix with size R^{m*n}

    A = np.empty([m,n])
    # ground truth
    z = 10*np.random.rand(n,1)
    # create noise vector with the length equals to m
    for i in range(m):
        for j in range(n):
            A[i,j] = np.random.normal(mu, 1)
        norm_A_i = np.linalg.norm(A[i,:],2)
        # scale each row of A such that the norm of each row is equal to 1.
        A[i,:] = A[i,:]/norm_A_i
    delta = np.random.normal(mu, sigma, size=(m,1))
    return A, z, delta

In [46]:
A, z, delta = data_generator(m,n,mu,sigma)
# generate observations
b = np.matmul(A,z)+delta
# check about A,z,b's shape
print(A.shape)
print(z.shape)
print(delta.shape)
print(b.shape)
print(np.linalg.norm(A[1,:],2))

(500, 50)
(50, 1)
(500, 1)
(500, 1)
1.0


**Implement a class for all attributes**\
lr: learning rate\
N: batch size\
theta: momentum parameter\
choic: 0 indicates "with replacement", 1 indicates "without replacement"\
T: epoch

In [87]:
class optimizer:
  '''
  lr: learning rate
  N: batch size
  theta: momentum parameter
  choic: 0 indicates "with replacement", 1 indicates "without replacement"
  T: epoch
  '''
  def __init__(self):
    self.lr = 0.1
    self.N = 1
    self.theta = 0.1
    self.choice = 0
    self.T = 10
    self.stop_criteria = 1e-1
  # set_ function are all functions to change the value.
  def set_lr(self,lr):
    self.lr = lr

  def set_N(self,N):
    self.N = N

  def set_theta(self,theta):
    self.theta = theta

  def set_choice(self,choice):
    self.choice = choice
  def set_T(self,T):
    self.T = T

  # define a function compute least square loss function value.
  def least_square(self,A,x,b):
    m,n = A.shape
    residual = np.matmul(A,x)-b
    loss = 0.5*(1/m)*np.linalg.norm(residual,2)**2
    return loss
  # define a function compute gradient

  def grad(self,A,x,b):
    residual = np.matmul(A,x)-b
    # gradient is A^T(Ax-b)
    grad = np.matmul(np.transpose(A),residual)
    return grad

  # define a function to sample the mini_batch of data.
  def mini_batch_sampling(self,A,b):
    m,n = A.shape
    # sampling with replacement
    if self.choice == 0:
      indx = np.random.choice(m, size=self.N, replace=True)
      batch_A = A[indx,:]
      batch_b = b[indx,:]
    # sampling without replacement
    if self.choice == 1:
      indx = np.random.choice(m, size=self.N, replace=False)
      np.random.shuffle(indx)
      batch_A = A[indx,:]
      batch_b = b[indx,:]
    return batch_A, batch_b

  # function to implement SGD.(for sampling with or without replacement)
  def sgd(self,A,b,x):
    m,n = A.shape
    count = 0
    stop = 100
    while (count <= self.T) and (stop >= self.stop_criteria):
      batch_A, batch_b = self.mini_batch_sampling(A,b)
      m1,n1 = batch_A.shape
      grad = (1/m1)*self.grad(batch_A,x,batch_b)
      stop = np.linalg.norm(grad)
      x += -self.lr*grad
      count += 1
    return x, count
  # function to implement sgd with average.
  def sgd_average(self,A,b,x):
    m,n = A.shape
    count = 0
    stop = 100
    x_list = []
    x_list.append(x)
    while (count <= self.T) and (stop >= self.stop_criteria):
      batch_A, batch_b = self.mini_batch_sampling(A,b)
      m1,n1 = batch_A.shape
      grad = (1/m1)*self.grad(batch_A,x,batch_b)
      stop = np.linalg.norm(grad)
      x += -self.lr*grad
      x_list.append(x)
      count += 1
      # return average
    return np.mean(np.array(x_list),axis=0), count
  # function to implement sgd with momentum acceleration.
  def sgd_momentum(self,A,b,x):
    count = 0
    m,n = A.shape
    momentum = np.zeros((n,1))
    stop = 100
    while (count <= self.T) and (stop >= self.stop_criteria):
      batch_A, batch_b = self.mini_batch_sampling(A,b)
      m1,n1 = batch_A.shape
      grad = (1/m1)*self.grad(batch_A,x,batch_b)
      stop = np.linalg.norm(grad)
      # momentum update
      momentum = self.theta*momentum - self.lr*grad
      x += momentum
      count += 1
    return x,count
  # function to implement gd with momentum.
  def gd_momentum(self,A,x,b):
    count = 0
    # initialization
    m,n = A.shape
    momentum =np.zeros((n,1))
    stop = 100
    grad = self.grad(A,x,b)
    while (count <= self.T) and (stop >= self.stop_criteria):
      grad = (1/m)*self.grad(A,x,b)
      stop = np.linalg.norm(grad)
      momentum = self.theta*momentum - self.lr*grad
      x += momentum
      count += 1
    return x,count



**Test case for simulation I**

In [89]:
# generate initial guess
# set class hyperparameter.
opt = optimizer()
opt.set_lr(1)
opt.set_N(10)
opt.set_T(10000)
opt.set_theta(0.5)
x0 = z+5*np.random.normal(0,1,size = (z.shape[0],z.shape[1]))

# with replacement
x_gd,iter_gd = opt.gd_momentum(A,x0,b)
obj_val0 = opt.least_square(A,x_gd,b)
sample_gd = m * iter_gd
print('gradient descent+momentum,objective value:%f,samples:%d'%(obj_val0,sample_gd))
opt.set_choice(0)
x_sgd,iter_sgd = opt.sgd(A,b,x0)
obj_val1 = opt.least_square(A,x_sgd,b)
sample_sgd = opt.N * iter_sgd
print('sgd sampling with replacement,objective value:%f,samples:%d'%(obj_val1,sample_sgd))

x_avg_sgd,iter_avg_sgd = opt.sgd_average(A,b,x0)
obj_val2 = opt.least_square(A,x_avg_sgd,b)
sample_sgd_avg = opt.N * iter_avg_sgd
print('sgd with replacement + averaging,objective value:%f,samples:%d'%(obj_val2,sample_sgd_avg))

x_sgd_mom,iter_sgd_mom = opt.sgd_momentum(A,b,x0)
obj_val3 = opt.least_square(A,x_sgd_mom,b)
sample_sgd_mom = opt.N * iter_sgd_mom
print('sgd with replacement + momentum,objective value:%f, samples: %d'%(obj_val3,sample_sgd_mom))


# without replacement
opt.set_choice(1)

x_sgd1,iter_sgd1 = opt.sgd(A,b,x0)
obj_val11 = opt.least_square(A,x_sgd,b)
sample_sgd1 = opt.N * iter_sgd1
print('sgd without replacement,objective value:%f,samples:%d'%(obj_val11,sample_sgd1))

x_avg_sgd1,iter_avg_sgd1 = opt.sgd_average(A,b,x0)
obj_val21 = opt.least_square(A,x_avg_sgd,b)
sample_sgd_avg1 = opt.N * iter_avg_sgd1
print('sgd without replacement + iterative averaging,objective value:%f,samples:%d'%(obj_val21,sample_sgd_avg1))

x_sgd_mom1,iter_sgd_mom1 = opt.sgd_momentum(A,b,x0)
obj_val31 = opt.least_square(A,x_sgd_mom,b)
sample_sgd_mom1 = opt.N * iter_sgd_mom1
print('sgd without replacement + momentum,objective value:%f,samples:%d'%(obj_val31,sample_sgd_mom1))



gradient descent+momentum,objective value:0.543773,samples:23000
sgd sampling with replacement,objective value:0.300546,samples:1010
sgd with replacement + averaging,objective value:0.285728,samples:2240
sgd with replacement + momentum,objective value:0.305379, samples: 6180
sgd without replacement,objective value:0.290193,samples:400
sgd without replacement + iterative averaging,objective value:0.285728,samples:280
sgd without replacement + momentum,objective value:0.295782,samples:3040


**Test case for simulation 2**

In [86]:
# Simulation 2
B_set = [1, 10, 50, 100, 150,200,250,300,350,400,500]
x0 = z+5*np.random.normal(0,1,size = (z.shape[0],z.shape[1]))
alpha = 2e-3
opt2 = optimizer()
opt2.set_choice(0)
for batch_size in B_set:
  opt2.set_N(batch_size)
  lr = batch_size*alpha
  opt2.set_lr(lr)
  x_sgd, iter_sgd = opt2.sgd(A,b, x0)
  sample_size = iter_sgd*batch_size
  obj_val = opt2.least_square(A,x_sgd,b)
  print('learning rate is: %f, objective value is :%f, sample size:%d'%(lr, obj_val,sample_size))

learning rate is: 0.002000, objective value is :11.836279, sample size:11
learning rate is: 0.020000, objective value is :11.723547, sample size:110
learning rate is: 0.100000, objective value is :11.183717, sample size:550
learning rate is: 0.200000, objective value is :10.130944, sample size:1100
learning rate is: 0.300000, objective value is :8.815433, sample size:1650
learning rate is: 0.400000, objective value is :7.325047, sample size:2200
learning rate is: 0.500000, objective value is :5.874710, sample size:2750
learning rate is: 0.600000, objective value is :4.533164, sample size:3300
learning rate is: 0.700000, objective value is :3.370679, sample size:3850
learning rate is: 0.800000, objective value is :2.483235, sample size:4400
learning rate is: 1.000000, objective value is :1.733151, sample size:5500
